# for this emailbuilder we will use edenAI

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.llms import EdenAI
import requests
import json
from types import SimpleNamespace
from sqlalchemy import create_engine, text

_ = load_dotenv(find_dotenv())
edenai_api_key = os.getenv("EDENAI_API_KEY")
DB_URL = os.getenv("DB_VM_URL")
engine = create_engine(DB_URL)
conn = engine.connect()

In [3]:
def get_answer(prompt):
    url = "https://api.edenai.run/v2/text/chat"

    payload = {
        "response_as_dict": True,
        "attributes_as_list": False,
        "show_original_response": False,
        "temperature": 0.0,
        "max_tokens": 400,
        "providers": "openai",
        "text": f"{prompt}",
    }
    headers = {
        "accept": "application/json",
        "content_type": "application/json",
        "authorization": f"Bearer {edenai_api_key}"
    }

    response = requests.post(url, json=payload, headers=headers)
    responseObject = json.loads(response.text, object_hook=lambda d: SimpleNamespace(**d))
    return responseObject.openai.generated_text

In [9]:
# TOEVOEGING

def get_email_prompts(email_list) -> list:

    email_string = ','.join(email_list)

    url = f'https://dep2-g1.vichogent.be/contact?ids={email_string}'

    response = requests.get(url)

    query_list = ["\'" + i + "\'" for i in email_list]
    query_string = ','.join(query_list)

    query = text(f"select ContactpersoonId,FunctieTitel from Contactfiche where ContactpersoonId in ({query_string})")
    result = conn.execute(query).fetchall()
    function_dict = {}
    for row in result:
        function_dict[row[0]] = row[1]


    email_prompts = []

    if response.status_code == 200:
        json_data = response.json()

        for user_data in json_data:
            user = user_data['contact']
            
            function = function_dict[user]
            
            campaigns = user_data['CampagneNaam']

            if campaigns:
                campaigns_list = ', '.join(campaigns.values())
                prompt = f"Create an email written for {user} that executes the following function: {function} and recommend the following campaigns: {campaigns_list}."
            else:
                prompt = f"Create an email written for {user} that executes the following function: {function} but there are no campaigns to recommend."

            email_prompts.append(prompt)

    else:
        error = f"Error: {response.status_code} - {response.text}"
        return list(error)
    return email_prompts
        
# EINDE TOEVOEGING

In [10]:
prompts = get_email_prompts(['C39094D7-60B1-EB11-811F-001DD8B72B62', '9D75B0F8-72D7-EB11-8121-001DD8B72B61'])

print(prompts)
user= "C39094D7-60B1-EB11-811F-001DD8B72B62"
function= 'Verantwoordelijke Personeel'
campaigns = ['D9834556-240C-EE11-8F6E-6045BD8956C9', 'B3865100-DEEF-ED11-8849-6045BD895233']

prompt= f"Create an email written for Erik Van Vliet that executes the following function: {function} and recommend the following campaigns: {campaigns}. You do not want an answert as this is an automated mail"

# text= f"Create an email written for {user} that executes the following function: {function} and recommend the following campaigns: {campaigns}"

# text= f"schrijf een email naar {user} die binnen het bedrijf de volgende functie heeft: {function} en biedt volgende campagnes aan: {campaigns}"

['Create an email written for 9D75B0F8-72D7-EB11-8121-001DD8B72B61 that executes the following function: Operations & Enzblement Lead but there are no campaigns to recommend.', 'Create an email written for C39094D7-60B1-EB11-811F-001DD8B72B62 that executes the following function: HR Director but there are no campaigns to recommend.']


In [8]:
dict = {}
for prompt in prompts:
    answer = get_answer(prompt)
    dict[prompt] = answer
    
for key, value in dict.items():
    print(key)
    print(value)
    print("--------------------------------------------------")
    print("---------------------------------------------")

Create an email written for 9D75B0F8-72D7-EB11-8121-001DD8B72B61 that executes the following function: Operations & Enzblement Lead but there are no campaigns to recommend.
Subject: Update on Operations & Enzblement Lead Role

Dear [9D75B0F8-72D7-EB11-8121-001DD8B72B61],

I hope this email finds you well. I am writing to provide you with an update regarding your role as the Operations & Enzblement Lead. 

Firstly, I would like to express my appreciation for your dedication and hard work in this position. Your contributions have been invaluable to our team, and we are grateful for your commitment to excellence.

However, I regret to inform you that currently, there are no campaigns available for you to recommend. Our team has thoroughly assessed the current projects and determined that there are no suitable campaigns that align with our objectives and requirements.

Please be assured that this situation is temporary, and we are actively working on identifying new opportunities for you t

In [ ]:
prompts = get_email_prompts(['C39094D7-60B1-EB11-811F-001DD8B72B62', '9D75B0F8-72D7-EB11-8121-001DD8B72B61'])